In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
import numpy as np
from itertools import combinations

In [ ]:
class OneVsOneClassifierCustom(BaseEstimator, ClassifierMixin):
    def __init__(self, base_model=LogisticRegression):
        self.base_model = base_model
        self.models = []
        self.class_pairs = []

    def fit(self, X, y):
        self.models = []
        self.class_pairs = []

        # Obtener clases únicas
        classes = np.unique(y)
        
        # Crear combinaciones de pares (0 vs 1), (0 vs 2), ...
        for (c1, c2) in combinations(classes, 2):
            # Filtrar dataset por solo esas dos clases
            idx = np.where((y == c1) | (y == c2))
            X_pair = X[idx]
            y_pair = y[idx]

            # Modelo nuevo
            model = self.base_model()
            model.fit(X_pair, y_pair)

            self.models.append(model)
            self.class_pairs.append((c1, c2))
        
        return self

    def predict(self, X):
        votes = np.zeros((X.shape[0], len(self.class_pairs)))
        
        predictions = []
        for i, model in enumerate(self.models):
            preds = model.predict(X)
            votes[:, i] = preds
        
        # Voto mayoritario por fila
        final_preds = []
        for row in votes:
            counts = np.bincount(row.astype(int))
            final_preds.append(np.argmax(counts))
        
        return np.array(final_preds)